# Knowledge Structure Graph Generator

## 1. DB연결

In [1]:
import pymysql

In [2]:
juso_db = pymysql.connect(
    user='root', 
    passwd='1234', 
    host='127.0.0.1', 
    db='test', 
    charset='utf8'
)

In [3]:
cursor = juso_db.cursor(pymysql.cursors.DictCursor)

In [4]:
sql = "SELECT * FROM knowledge_structure;"
cursor.execute(sql)
result = cursor.fetchall()

In [5]:
import pandas as pd

result = pd.DataFrame(result)
print("------------------------------지식구조 목록 리스트------------------------------")
print(result) #지식구조 목록 리스트화

------------------------------지식구조 목록 리스트------------------------------
       DOC_ID COT_MODE                                KNOWLEDGE_STRUCTURE
0        hemp      scs  sourc,origin,1.0\nsourc,millennium,1.0\nsourc,...
1          감초      scs  함량,재배,2.9\n효능,역할,3.21\n하루,이상,3.9\n이상,단맛,3.9\n한...
2          녹차      scs  순수,티백,1.52\n제품,제조,2.15\n효과,흥분,2.95\n제품,순수,2.95...
3          대마      scs  마약,법률,3.06\n약물,중독,3.26\n한국,미국,3.87\n필로폰,헤로인,4....
4         바이오      scs  소재,개발,3.07\n분야,성과,3.6\n기술,식품,3.71\n분야,식품,3.79\...
5    인공지능_이루다      scs  제공,특정,2.76\n대화,연인들,3.54\n개발사,여성,3.54\n말투,학습,4....
6   인공지능_이루다2      scs  특정,자사,1.63\n수집,변호사,2.1\n개인정보,유출,2.76\n개인정보,수집,...
7     인공지능_제조      scs  제조,역할,1.0\n제조,육성,1.0\n역할,육성,1.0\n산업,제조,1.34\n산...
8     인공지능_호텔      scs  호텔,서비스,1.99\n코로나,업계,2.76\n제공,정보,2.98\n제공,차량,2....
9           차      scs  남부,지방,2.71\n재배,지역,4.03\n식물,우려,4.81\n남부,지역,4.85...
10          콩      scs  두부,된장,1.8\n고기,짜파게티,2.53\n여성,에스트로겐,3.0\n성분,여성호르...
11         홍차      scs  여성,커피,2.96\n시간,우

In [6]:
knowledge_structure = result['KNOWLEDGE_STRUCTURE']

## 2. 데이터 선택

#### 작성할 파일 제목 정하기
* file_name = "파일명"

In [92]:
file_name = "녹차" 

#### 추출하기 원하는 인덱스 선정

In [93]:
data = knowledge_structure[2] #인덱스만 변경하기
print(data)

순수,티백,1.52
제품,제조,2.15
효과,흥분,2.95
제품,순수,2.95
녹차,제품,2.98
녹차,현미,3.23
테아닌,효과,3.76
테아닌,홍차,4.0
카페인,커피,4.06
카페인,효과,4.11
소비,티백,4.17
카페인,결과,4.23
테아닌,성분,4.46
테아닌,함유,4.68
결과,하루,5.0
녹차,홍차,5.21
소비,차나무,5.27
녹차,이름,6.11



In [94]:
import re

contents = re.split('\n', data)
contents = contents[:-1] #맨 끝 데이터는 공백

In [95]:
#노드 추가
for content in contents:
    value = re.split(',', content) # ,단위로 구분하여 리스트화
    print(value)

['순수', '티백', '1.52']
['제품', '제조', '2.15']
['효과', '흥분', '2.95']
['제품', '순수', '2.95']
['녹차', '제품', '2.98']
['녹차', '현미', '3.23']
['테아닌', '효과', '3.76']
['테아닌', '홍차', '4.0']
['카페인', '커피', '4.06']
['카페인', '효과', '4.11']
['소비', '티백', '4.17']
['카페인', '결과', '4.23']
['테아닌', '성분', '4.46']
['테아닌', '함유', '4.68']
['결과', '하루', '5.0']
['녹차', '홍차', '5.21']
['소비', '차나무', '5.27']
['녹차', '이름', '6.11']


## 3. Data JavaScript File 생성

### 파일 2개 생성
* keyword.js : data(nodes, edges)
* keyword.html : graph

### 01. keyword.js

In [96]:
import networkx as nx

In [97]:
G = nx.Graph()

In [98]:
# 에지리스트를 그래프에 넣기 : 그래프에 입력된 노드 확인용
for content in contents:
    tmp = re.split(',', content)
    G.add_edge(tmp[0], tmp[1], weight = tmp[2])

In [99]:
degree_list = [degree for degree in G.degree()] # 각 노드별 차수 저장

In [100]:
# 파일 저장용 nodes, edges
graph_nodes = "var nodes = "
graph_edges = "var edges = "

# 리스트 생성용
num = 0
nodes_list = []
reverse_graph = dict()

for name in G.nodes():
    # 딕셔너리 1개씩 생성하여 넣기
    tmp = dict()
    tmp['id'] = num
    tmp['label'] = name
    tmp['degree'] = degree_list[num][1]

    
    # 색상은 파랑색으로 지정
    tmp['color'] = "#97C2FC"
    
#     # 차수가 3이상이면 빨간색 지정
#     if(degree_list[num][1] >= 3):
#         tmp['color'] = "#FB7E81"
#     else:
#         tmp['color'] = "#97C2FC"
        
    nodes_list.append(tmp)   # {id: 0, label: 'hemp'}형태
    
    reverse_graph[name] = num   # {'hemp':0, 'cannabi':1}형태, 목적: 'hemp'검색시 바로 노드 번호 확인(편의성 목적)
    num += 1

In [101]:
nodes_list

[{'id': 0, 'label': '순수', 'degree': 2, 'color': '#97C2FC'},
 {'id': 1, 'label': '티백', 'degree': 2, 'color': '#97C2FC'},
 {'id': 2, 'label': '제품', 'degree': 3, 'color': '#97C2FC'},
 {'id': 3, 'label': '제조', 'degree': 1, 'color': '#97C2FC'},
 {'id': 4, 'label': '효과', 'degree': 3, 'color': '#97C2FC'},
 {'id': 5, 'label': '흥분', 'degree': 1, 'color': '#97C2FC'},
 {'id': 6, 'label': '녹차', 'degree': 4, 'color': '#97C2FC'},
 {'id': 7, 'label': '현미', 'degree': 1, 'color': '#97C2FC'},
 {'id': 8, 'label': '테아닌', 'degree': 4, 'color': '#97C2FC'},
 {'id': 9, 'label': '홍차', 'degree': 2, 'color': '#97C2FC'},
 {'id': 10, 'label': '카페인', 'degree': 3, 'color': '#97C2FC'},
 {'id': 11, 'label': '커피', 'degree': 1, 'color': '#97C2FC'},
 {'id': 12, 'label': '소비', 'degree': 2, 'color': '#97C2FC'},
 {'id': 13, 'label': '결과', 'degree': 2, 'color': '#97C2FC'},
 {'id': 14, 'label': '성분', 'degree': 1, 'color': '#97C2FC'},
 {'id': 15, 'label': '함유', 'degree': 1, 'color': '#97C2FC'},
 {'id': 16, 'label': '하루', 'degr

In [102]:
graph_nodes += str(nodes_list) + ";"
print(graph_nodes)

var nodes = [{'id': 0, 'label': '순수', 'degree': 2, 'color': '#97C2FC'}, {'id': 1, 'label': '티백', 'degree': 2, 'color': '#97C2FC'}, {'id': 2, 'label': '제품', 'degree': 3, 'color': '#97C2FC'}, {'id': 3, 'label': '제조', 'degree': 1, 'color': '#97C2FC'}, {'id': 4, 'label': '효과', 'degree': 3, 'color': '#97C2FC'}, {'id': 5, 'label': '흥분', 'degree': 1, 'color': '#97C2FC'}, {'id': 6, 'label': '녹차', 'degree': 4, 'color': '#97C2FC'}, {'id': 7, 'label': '현미', 'degree': 1, 'color': '#97C2FC'}, {'id': 8, 'label': '테아닌', 'degree': 4, 'color': '#97C2FC'}, {'id': 9, 'label': '홍차', 'degree': 2, 'color': '#97C2FC'}, {'id': 10, 'label': '카페인', 'degree': 3, 'color': '#97C2FC'}, {'id': 11, 'label': '커피', 'degree': 1, 'color': '#97C2FC'}, {'id': 12, 'label': '소비', 'degree': 2, 'color': '#97C2FC'}, {'id': 13, 'label': '결과', 'degree': 2, 'color': '#97C2FC'}, {'id': 14, 'label': '성분', 'degree': 1, 'color': '#97C2FC'}, {'id': 15, 'label': '함유', 'degree': 1, 'color': '#97C2FC'}, {'id': 16, 'label': '하루', 'degree':

In [103]:
print(reverse_graph) # graph_nodes를 뒤집어서 저장, 예를 들어  label명 : id명

{'순수': 0, '티백': 1, '제품': 2, '제조': 3, '효과': 4, '흥분': 5, '녹차': 6, '현미': 7, '테아닌': 8, '홍차': 9, '카페인': 10, '커피': 11, '소비': 12, '결과': 13, '성분': 14, '함유': 15, '하루': 16, '차나무': 17, '이름': 18}


In [104]:
edges_list = []

edge_max = 0.0
edge_min = 7.0

for edge in G.edges(data='weight'):
#     print("step: " + edge[2])
    tmp = float(edge[2])

    #최댓값 저장
    if(edge_max <= tmp):
        edge_max = tmp
    
    #최솟값 저장
    if(edge_min >= tmp):
        edge_min = tmp
        
#     print("edge_max: " + str(edge_max))
#     print("edge_min: " + str(edge_min))
#     print()

In [105]:
# edges는 {'hemp', 'cannabi', '3.21'}형태
for edge in G.edges(data='weight'):
    tmp = dict()
    tmp['from']   = reverse_graph.get(edge[0])     # 'hemp'검색시 인덱스값
    tmp['to']     = reverse_graph.get(edge[1])     # 'cannabi'검색시 인덱스값
    tmp['label']  = edge[2]                        # 가중치 저장
    
    length = 7.0 - float(edge[2])
    convert_edge_min = 7.0 - edge_max
    minmax_scale = ((length - convert_edge_min) / (edge_max - edge_min)) * 500        # 길이를 min max 스케일링
    tmp['value']  = round(minmax_scale, 2)         # Edge 두께를 가중치의 크기에 따라 변경(숫자가 클수록 두껍게)
    
    length = float(edge[2])
    minmax_scale = ((length - edge_min)/(edge_max - edge_min)) * 500
    tmp['length'] = round(minmax_scale +220, 2)    # Edge 길이를 min max 스케일링
    
    tmp['color'] = "#d3d3d3"                      

    edges_list.append(tmp)

In [106]:
graph_edges += str(edges_list) + ";"
print(graph_edges)

var edges = [{'from': 0, 'to': 1, 'label': '1.52', 'value': 500.0, 'length': 220.0, 'color': '#d3d3d3'}, {'from': 0, 'to': 2, 'label': '2.95', 'value': 344.23, 'length': 375.77, 'color': '#d3d3d3'}, {'from': 1, 'to': 12, 'label': '4.17', 'value': 211.33, 'length': 508.67, 'color': '#d3d3d3'}, {'from': 2, 'to': 3, 'label': '2.15', 'value': 431.37, 'length': 288.63, 'color': '#d3d3d3'}, {'from': 2, 'to': 6, 'label': '2.98', 'value': 340.96, 'length': 379.04, 'color': '#d3d3d3'}, {'from': 4, 'to': 5, 'label': '2.95', 'value': 344.23, 'length': 375.77, 'color': '#d3d3d3'}, {'from': 4, 'to': 8, 'label': '3.76', 'value': 255.99, 'length': 464.01, 'color': '#d3d3d3'}, {'from': 4, 'to': 10, 'label': '4.11', 'value': 217.86, 'length': 502.14, 'color': '#d3d3d3'}, {'from': 6, 'to': 7, 'label': '3.23', 'value': 313.73, 'length': 406.27, 'color': '#d3d3d3'}, {'from': 6, 'to': 9, 'label': '5.21', 'value': 98.04, 'length': 621.96, 'color': '#d3d3d3'}, {'from': 6, 'to': 18, 'label': '6.11', 'value': 

In [107]:
graph_data = graph_nodes + "\n\n" + graph_edges

In [108]:
print(graph_data)

var nodes = [{'id': 0, 'label': '순수', 'degree': 2, 'color': '#97C2FC'}, {'id': 1, 'label': '티백', 'degree': 2, 'color': '#97C2FC'}, {'id': 2, 'label': '제품', 'degree': 3, 'color': '#97C2FC'}, {'id': 3, 'label': '제조', 'degree': 1, 'color': '#97C2FC'}, {'id': 4, 'label': '효과', 'degree': 3, 'color': '#97C2FC'}, {'id': 5, 'label': '흥분', 'degree': 1, 'color': '#97C2FC'}, {'id': 6, 'label': '녹차', 'degree': 4, 'color': '#97C2FC'}, {'id': 7, 'label': '현미', 'degree': 1, 'color': '#97C2FC'}, {'id': 8, 'label': '테아닌', 'degree': 4, 'color': '#97C2FC'}, {'id': 9, 'label': '홍차', 'degree': 2, 'color': '#97C2FC'}, {'id': 10, 'label': '카페인', 'degree': 3, 'color': '#97C2FC'}, {'id': 11, 'label': '커피', 'degree': 1, 'color': '#97C2FC'}, {'id': 12, 'label': '소비', 'degree': 2, 'color': '#97C2FC'}, {'id': 13, 'label': '결과', 'degree': 2, 'color': '#97C2FC'}, {'id': 14, 'label': '성분', 'degree': 1, 'color': '#97C2FC'}, {'id': 15, 'label': '함유', 'degree': 1, 'color': '#97C2FC'}, {'id': 16, 'label': '하루', 'degree':

In [109]:
f = open("./results/result_webgraph/data/" + file_name + ".js", 'w', encoding = 'utf8')
f.write(graph_data)
f.close()
print(file_name + ".js 파일 생성 완료")

녹차.js 파일 생성 완료


### 2. keyword.html

In [110]:
html_top = """
<!doctype html>
<html>
<head>
	<title>Knowledge Structure</title>
    """

In [111]:
vis_location = """<script type="text/javascript" src="../../lib/drawing/vis.js"></script>
    """

In [112]:
js_location = """<script src="../../results/result_webgraph/data/""" + file_name + """.js" charset="utf-8">
    </script>
    <style type="text/css">#mynetwork 
        {width: 100%;height: 1000px;border: 1px solid lightgray;}
    </style>
</head>
<body>
    """

In [113]:
title = """<p> Cheonsol of Knowledge Structure.</p>"""

In [114]:
body = """
    <div id="mynetwork"></div>
	<script type="text/javascript">
		var container = document.getElementById('mynetwork');
		var data = { nodes: nodes, edges: edges }; """

In [115]:
option = """
        var options = 
        { "nodes": {"shape" : "circle",
                    "font":{"size": 35}},
        "edges": {"scaling":{"max":70}, "font": {"size" : 40}, 
            "smooth": {"type":"continuous","roundness": 0.5}},
            "physics": {"barnesHut": {"centralGravity": 0.05},
            "minVelocity": 0.75}};
        """

In [116]:
bottom = """var network = new vis.Network(container, data, options);
	</script>
</body>
</html>
"""

In [117]:
html_text = html_top + vis_location + js_location + title + body + option + bottom

In [118]:
html_text

'\n<!doctype html>\n<html>\n<head>\n\t<title>Knowledge Structure</title>\n    <script type="text/javascript" src="../../lib/drawing/vis.js"></script>\n    <script src="../../results/result_webgraph/data/녹차.js" charset="utf-8">\n    </script>\n    <style type="text/css">#mynetwork \n        {width: 100%;height: 1000px;border: 1px solid lightgray;}\n    </style>\n</head>\n<body>\n    <p> Cheonsol of Knowledge Structure.</p>\n    <div id="mynetwork"></div>\n\t<script type="text/javascript">\n\t\tvar container = document.getElementById(\'mynetwork\');\n\t\tvar data = { nodes: nodes, edges: edges }; \n        var options = \n        { "nodes": {"shape" : "circle",\n                    "font":{"size": 35}},\n        "edges": {"scaling":{"max":70}, "font": {"size" : 40}, \n            "smooth": {"type":"continuous","roundness": 0.5}},\n            "physics": {"barnesHut": {"centralGravity": 0.05},\n            "minVelocity": 0.75}};\n        var network = new vis.Network(container, data, opti

In [119]:
html_file = open('./results/result_webgraph/' + file_name + '.html', 'w')
html_file.write(html_text)
html_file.close()
print(file_name + ".html 파일 생성 완료")

녹차.html 파일 생성 완료
